In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
x = tf.placeholder(tf.float32,[None,784], name="X")
y = tf.placeholder(tf.float32,[None,10], name="Y")

In [3]:
# 隐藏层
H1_NN = 256
W1 = tf.Variable(tf.random_normal([784,H1_NN]))
b1 = tf.Variable(tf.zeros([H1_NN]))

Y1 = tf.nn.relu(tf.matmul(x, W1) + b1)

In [4]:
# 输出层
W2 = tf.Variable(tf.random_normal([H1_NN, 10]))
b2 = tf.Variable(tf.zeros([10]))
forward = tf.matmul(Y1, W2) + b2

pred = tf.nn.softmax(forward)

In [5]:
# 交叉熵
#loss_function = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred),reduction_indices=1))

In [6]:
# 修改
# TensorFlow提供了softmax_cross_entropy_with_logits函数
# 用于避免因为log(0)值为NaN造成的数据不稳定
loss_function = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=forward,labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
train_epochs = 40
learning_rate = 0.01
display_step = 1 # 粒度
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)#一轮的训练有多少批次

In [8]:
# 优化器
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

In [9]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(pred,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [10]:
# train first
from time import time
startTime = time()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs,ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x:xs,y:ys})
    
    loss,acc = sess.run([loss_function,accuracy],
                       feed_dict={x:mnist.validation.images,
                                 y:mnist.validation.labels})
    

    if (epoch+1) % display_step == 0:
        print("Train Epoch:","%02d"%(epoch+1),
             "Loss=","{:.9f}".format(loss),"Accuracy=","{:.4f}".format(acc))
duration = time()-startTime
print("Train Finished takes:","{:.2f}".format(duration))

Train Epoch: 01 Loss= 1.287822604 Accuracy= 0.9424
Train Epoch: 02 Loss= 0.814032495 Accuracy= 0.9514
Train Epoch: 03 Loss= 0.626807809 Accuracy= 0.9534
Train Epoch: 04 Loss= 0.460729301 Accuracy= 0.9514
Train Epoch: 05 Loss= 0.478529871 Accuracy= 0.9510
Train Epoch: 06 Loss= 0.440502822 Accuracy= 0.9604
Train Epoch: 07 Loss= 0.451307088 Accuracy= 0.9572
Train Epoch: 08 Loss= 0.420605212 Accuracy= 0.9594
Train Epoch: 09 Loss= 0.436314940 Accuracy= 0.9668
Train Epoch: 10 Loss= 0.375402421 Accuracy= 0.9642
Train Epoch: 11 Loss= 0.320113093 Accuracy= 0.9672
Train Epoch: 12 Loss= 0.425253600 Accuracy= 0.9632
Train Epoch: 13 Loss= 0.485091299 Accuracy= 0.9644
Train Epoch: 14 Loss= 0.435038000 Accuracy= 0.9658
Train Epoch: 15 Loss= 0.438906252 Accuracy= 0.9666
Train Epoch: 16 Loss= 0.597189426 Accuracy= 0.9658
Train Epoch: 17 Loss= 0.413144767 Accuracy= 0.9712
Train Epoch: 18 Loss= 0.473069936 Accuracy= 0.9680
Train Epoch: 19 Loss= 0.495463729 Accuracy= 0.9696
Train Epoch: 20 Loss= 0.4946227

In [11]:
# 模型评估
accu_test = sess.run(
    accuracy,feed_dict={x:mnist.test.images, y:mnist.test.labels})
print("Test Accuracy: ",accu_test)

Test Accuracy:  0.9714


In [12]:
# 模型应用
# 预测
# 由于pred的预测结果是one-hot编码格式，所以需要转换为0-9数字
prediction_result = sess.run(tf.argmax(pred, 1),
                            feed_dict={x:mnist.test.images})

In [13]:
prediction_result[0:10]

array([7, 2, 1, 0, 4, 1, 4, 9, 6, 9])

In [18]:
import numpy as np
compare_lists = prediction_result==np.argmax(mnist.test.labels,1)
print(compare_list)

[ True  True  True ...  True  True  True]


In [19]:
err_lists = [i for i in range(len(compare_list)) 
             if compare_lists[i]==False]
print(err_lists,len(err_lists))

[8, 65, 149, 233, 247, 321, 340, 445, 449, 456, 460, 522, 552, 582, 619, 629, 646, 659, 674, 684, 691, 720, 740, 810, 842, 844, 882, 900, 924, 938, 947, 951, 956, 959, 965, 1000, 1014, 1032, 1039, 1107, 1125, 1156, 1181, 1185, 1226, 1232, 1242, 1247, 1289, 1312, 1319, 1325, 1328, 1331, 1393, 1425, 1494, 1522, 1527, 1530, 1549, 1609, 1681, 1717, 1718, 1721, 1737, 1751, 1754, 1781, 1800, 1865, 1878, 1901, 1968, 1981, 2040, 2053, 2098, 2109, 2125, 2130, 2135, 2182, 2185, 2215, 2225, 2272, 2293, 2325, 2369, 2387, 2445, 2447, 2454, 2462, 2597, 2604, 2631, 2648, 2654, 2720, 2721, 2739, 2770, 2810, 2863, 2896, 2921, 2927, 2939, 2953, 3005, 3012, 3023, 3060, 3073, 3117, 3130, 3225, 3250, 3263, 3289, 3336, 3337, 3376, 3490, 3503, 3520, 3558, 3567, 3597, 3674, 3681, 3688, 3702, 3726, 3727, 3767, 3776, 3780, 3796, 3808, 3811, 3817, 3838, 3850, 3876, 3893, 3902, 3906, 3941, 3976, 3985, 4065, 4078, 4176, 4177, 4201, 4212, 4224, 4237, 4248, 4271, 4289, 4350, 4359, 4400, 4425, 4433, 4497, 4504, 4536,

In [21]:
import numpy as np
def print_predict_errs(labels,
                       prediction):
    count = 0
    compare_lists = (prediction==np.argmax(labels,1))
    err_lists = [i for i in range(len(compare_lists)) if compare_lists[i]==False]
    for x in err_lists:
        print("index="+str(x)+
             "标签值=",np.argmax(labels[x]),
             "预测值=",prediction[x])
        count = count + 1
    print("总计"+str(count))
print_predict_errs(labels=mnist.test.labels,
                  prediction=prediction_result)


index=8标签值= 5 预测值= 6
index=65标签值= 4 预测值= 9
index=149标签值= 2 预测值= 3
index=233标签值= 8 预测值= 9
index=247标签值= 4 预测值= 2
index=321标签值= 2 预测值= 7
index=340标签值= 5 预测值= 3
index=445标签值= 6 预测值= 0
index=449标签值= 3 预测值= 5
index=456标签值= 2 预测值= 1
index=460标签值= 5 预测值= 9
index=522标签值= 7 预测值= 3
index=552标签值= 0 预测值= 8
index=582标签值= 8 预测值= 2
index=619标签值= 1 预测值= 8
index=629标签值= 2 预测值= 6
index=646标签值= 2 预测值= 6
index=659标签值= 2 预测值= 3
index=674标签值= 5 预测值= 3
index=684标签值= 7 预测值= 3
index=691标签值= 8 预测值= 9
index=720标签值= 5 预测值= 8
index=740标签值= 4 预测值= 9
index=810标签值= 7 预测值= 2
index=842标签值= 7 预测值= 2
index=844标签值= 8 预测值= 7
index=882标签值= 9 预测值= 4
index=900标签值= 1 预测值= 3
index=924标签值= 2 预测值= 7
index=938标签值= 3 预测值= 5
index=947标签值= 8 预测值= 4
index=951标签值= 5 预测值= 4
index=956标签值= 1 预测值= 2
index=959标签值= 4 预测值= 9
index=965标签值= 6 预测值= 0
index=1000标签值= 9 预测值= 3
index=1014标签值= 6 预测值= 0
index=1032标签值= 5 预测值= 6
index=1039标签值= 7 预测值= 1
index=1107标签值= 9 预测值= 3
index=1125标签值= 8 预测值= 9
index=1156标签值= 7 预测值= 8
index=1181标签值= 6 预测值= 1
index=